Reference : https://lsjsj92.tistory.com/568

# NMF 기반 DKT 문제 분석
- 🦆 개요 (Overview)
- ⚙️ 환경 설정 및 전처리 (Environment Setting & Preprocessing)
- 🎯 NMF 분석
- 🎯 예측 및 평가

## 🦆개요 (Overview)

### DKT, RS, MF Review

- 미션 8 내용 참조
- DKT (Deep Knowledge Tracing) : 사용자의 학습 수준을 추적하기 위한 문제로, 주로 문제에 대해 대상자가 풀 수 있을지 여부를 예측하는 문제
- RS (Recommendation System) : 사용자에게 아이템을 추천하는 문제로, 아이템에 대한 사용자의 선호여부, 선호도 (취향) 를 예측. '선호'를 문제를 맞출 가능성으로 보면 DKT 문제로 치환 가능
- MF (Matrix Factorization) : 행렬 분해 기법으로, 행렬로 표현되는 특성 벡터를 잠재 공간에 투영하는 투영행렬을 구하는데 사용. 투영을 통해 사용자의 아이템에 대한 선호를 잠재 factor에 대한 선호로 치환하여 RS 문제를 추정 가능


### 비음수 행렬 분해 (Non-negative matrix factorization: NMF) 를 통한 MF

> 비음수 행렬 분해(Non-negative matrix factorization, NMF)는 음수를 포함하지 않은 행렬 V를 음수를 포함하지 않은 행렬 W와 H의 곱으로 분해하는 알고리즘이다.[1] 행렬이 음수를 포함하지 않는 성질은 분해 결과 행렬을 찾기 쉽게 만든다. 일반적으로 행렬 분해는 정확한 해가 없기 때문에 이 알고리즘은 대략적인 해를 구하게 된다. 비음수 행렬 분해는 컴퓨터 시각 처리, 문서 분류, 음파 분석, 계량분석화학, 추천 시스템 등에 쓰인다.[(Wikipedia)](https://ko.wikipedia.org/wiki/%EB%B9%84%EC%9D%8C%EC%88%98_%ED%96%89%EB%A0%AC_%EB%B6%84%ED%95%B4)

- $V=WH+U$ 가 성립하는 W, H를 구하며, 오차 편향 U를 최소화한다
- $rank(V) = rank_+(V)$ 일때 분해를 NMF라고 한다 ($rank_+(V)$는 Non-negative rank)
- Non-negative Rank : $rank_+(V)=min\{q| \Sigma_{j=1}^{q}R_j=A, rank(R_1)=...=rand(R_q)=1, R_j \geq 0, \forall{j}\}$ ([참고](https://en.wikipedia.org/wiki/Nonnegative_rank_(linear_algebra)))
- Positive (Non-negative) Matrix : $r_{i,j} \geq 0, \forall{i,j} \implies R \geq 0$  ([참고](https://en.wikipedia.org/wiki/Nonnegative_matrix))


### 실습 진행 내용

- 환경 설정 및 전처리 : DKT 문제의 데이터를 협업 필터링에 맞는 형태로 변형하는 방법 설명
- NMF 분석 : NMF의 개념과 이를 통해 데이터를 분석하는 방법 설명
- 예측및 평가 : NMF 분석으로 생성된 변환 행렬을 통해 DKT 문제를 추론하는 방법 설명

## ⚙️ 환경 설정 및 전처리 (Environment Setting & Preprocessing)

### Loading Library

In [99]:
import numpy as np # linear algebra
import pandas as pd
import random
from sklearn.decomposition import NMF
from sklearn.metrics import accuracy_score, roc_auc_score
import os

### Data Load

In [100]:
data_dir='/opt/ml/input/data'
train_data = pd.read_csv(os.path.join(data_dir,'train_data.csv'))
test_data  = pd.read_csv(os.path.join(data_dir, 'test_data.csv'))

### 데이터 구성

- 데이터는 학습 데이터셋과 테스트 데이터셋으로 구분되어 있다.
- 각 데이터에는 userID, assessmentItemID, testId, answerCode, Timestamp, KnowledgeTag의 정보가 있다.
- 여기서 assessmentItemID는 문제의 고유 ID이며, answerCode는 사용자가 해당 문제의 정답을 맞췄는지 여부로, 맞췄으면 1, 틀렸으면 0으로 표기된다.
- 기본적인 협업 필터링 적용을 위해 본 실습에서는 userID, assessmentItemID, answerCode만을 사용한다.

In [101]:
userid, itemid = list(set(train_data.userID)), list(set(train_data.assessmentItemID))
n_user, n_item = len(userid), len(itemid)

print(f"Train dataset")
display(train_data.head(5))
print(f" Num. Users    : {n_user}")
print(f" Max. UserID   : {max(userid)}")
print(f" Num. Items    : {n_item}")
print(f" Num. Records  : {len(train_data)}")

userid, itemid = list(set(test_data.userID)), list(set(test_data.assessmentItemID))
n_user, n_item = len(userid), len(itemid)

print(f"Test dataset")
display(test_data.head(5))
print(f" Num. Users    : {n_user}")
print(f" Max. UserID   : {max(userid)}")
print(f" Num. Items    : {n_item}")
print(f" Num. Records  : {len(test_data)}")

Train dataset


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225


 Num. Users    : 6698
 Max. UserID   : 7441
 Num. Items    : 9454
 Num. Records  : 2266586
Test dataset


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,3,A050023001,A050000023,1,2020-01-09 10:56:31,2626
1,3,A050023002,A050000023,1,2020-01-09 10:56:57,2626
2,3,A050023003,A050000023,0,2020-01-09 10:58:31,2625
3,3,A050023004,A050000023,0,2020-01-09 10:58:36,2625
4,3,A050023006,A050000023,0,2020-01-09 10:58:43,2623


 Num. Users    : 744
 Max. UserID   : 7439
 Num. Items    : 9454
 Num. Records  : 260114


In [102]:
display(test_data.tail(3))

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244
260113,7439,A040130005,A040000130,-1,2020-10-14 23:10:03,8832


위와 같이 테스트 데이터셋에서는 answerCode가 -1인 경우가 나타난다. 이는 평가를 위한 것으로 해당 레코드는 제외하고 실습을 수행한다.

### Data Preprocessing

중복 레코드 제거
 - RS 모델에서는 시간에 따른 변화를 고려하지 않기 때문에 최종 성적만을 바탕으로 평가한다.
 - 사용자+문제항목을 Unique key로 하여 최종 레코드만을 보존하고 나머지 제거한다.

In [103]:
train_data.drop_duplicates(subset = ["userID", "assessmentItemID"],
                     keep = "last", inplace = True)
test_data.drop_duplicates(subset = ["userID", "assessmentItemID"],
                     keep = "last", inplace = True)

불필요한 column 제거
- 다음과 같이 pandas에서는 불필요한 column을 제거할 수 있다.

In [104]:
train_data.drop(['Timestamp','testId','KnowledgeTag'],
                axis=1, inplace=True, errors='ignore')
train_data.head(10)

,userID,assessmentItemID,answerCode
0,0,A060001001,1
1,0,A060001002,1
2,0,A060001003,1
3,0,A060001004,1
4,0,A060001005,1
5,0,A060001007,1
6,0,A060003001,0
7,0,A060003002,1
8,0,A060003003,1
9,0,A060003004,1


평가 항목 제거
- 테스트 데이터셋에서 answerCode가 -1인 항목은 최종 평가시 사용되는 항목으로 여기에선 사용할 수 없다.
- 아래 결과에서와 같이 User, Item 수는 변화 없이 총 레코드 수만 변한다.

In [106]:
test_data_old = test_data.copy()
n_user_old, n_item_old = n_user, n_item

real_eval_data=test_data[test_data.answerCode<0].copy()
test_data  = test_data[test_data.answerCode>=0].copy()

userid, itemid = list(set(test_data.userID)), list(set(test_data.assessmentItemID))
n_user, n_item = len(userid), len(itemid)

display(test_data.tail(5))
print(f" Num. Users    : {n_user}->{n_user}")
print(f" Max. UserID   : {max(userid)}")
print(f" Num. Items    : {n_item}->{n_item}")
print(f" Num. Records  : {len(test_data_old)}->{len(test_data)}")

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244


 Num. Users    : 744->744
 Max. UserID   : 7439
 Num. Items    : 9454->9454
 Num. Records  : 256073->255329


평가 항목 신규 생성
- 남은 테스트 항목 중, 각 사용자별 최종 레코드를 새로운 평가 항목으로 정한다.

In [108]:
eval_data = test_data.copy()
eval_data.drop_duplicates(subset = ["userID"],
                     keep = "last", inplace = True)
display(eval_data.head(5))
display(eval_data.tail(5))
print(f" Num. Records  : {len(eval_data)}")

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
1034,3,A050133007,A050000133,0,2020-10-26 13:13:11,5289
1705,4,A070146007,A070000146,1,2020-12-27 02:47:31,9080
3022,13,A070111007,A070000111,1,2020-12-27 04:35:01,9660
4282,17,A090064005,A090000064,1,2020-10-30 05:47:22,2611
4669,26,A060135006,A060000135,0,2020-10-23 11:44:01,1422


,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
260051,7395,A040122004,A040000122,0,2020-09-08 02:05:18,2102
260066,7404,A030111004,A030000111,1,2020-10-13 09:47:31,7636
260081,7416,A050193003,A050000193,0,2020-10-04 02:44:17,10402
260096,7417,A050193003,A050000193,0,2020-09-06 13:08:54,10402
260112,7439,A040130004,A040000130,1,2020-10-14 23:09:31,8244


 Num. Records  : 744


평가 항목을 테스트 항목에서 제거한다.

In [109]:
real_test_data=test_data.copy()

In [110]:
test_data.drop(index=eval_data.index, inplace=True, errors='ignore')
display(test_data.tail(5))
print(f" Num. Records  : {len(test_data)}")

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
260107,7439,A040197005,A040000197,0,2020-08-21 07:39:40,2132
260108,7439,A040197006,A040000197,1,2020-08-21 07:39:45,2132
260109,7439,A040130001,A040000130,0,2020-10-14 23:07:23,8832
260110,7439,A040130002,A040000130,1,2020-10-14 23:07:41,8832
260111,7439,A040130003,A040000130,1,2020-10-14 23:08:02,8244


 Num. Records  : 254585


사용자 - 문제항목 관계를 pivot 테이블로 변경
 - 각 사용자별로 해당 문제를 맞췄는지 여부를 matrix 형태로 변경
 - 해당 문제를 푼 적이 없는 경우 0.5(예시)으로 설정

In [111]:
matrix_train = train_data.pivot_table('answerCode', index='userID', columns='assessmentItemID')
matrix_train.fillna(0.5, inplace=True)
display(matrix_train.head(5))
print(f"Result Shape is {matrix_train.shape}")

assessmentItemID,A010001001,A010001002,A010001003,A010001004,A010001005,A010002001,A010002002,A010002003,A010002004,A010002005,...,A090073003,A090073004,A090073005,A090073006,A090074001,A090074002,A090074003,A090074004,A090074005,A090074006
userID,,,,,,,,,,,,,,,,,,,,,
0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
6,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


Result Shape is (6698, 9454)


## 🎯 NMF 분석

### 데이터 인덱스 매핑 생성

사용자/문제항목 ID와 table상에서의 index를 매칭시키기 위한 lookup table을 dictionary 형태로 생성

In [112]:
user_id2idx = {v:i for i,v in enumerate(matrix_train.index)}
user_idx2id = {i:v for i,v in enumerate(matrix_train.index)}

item_id2idx = {v:i for i,v in enumerate(matrix_train.columns)}
item_idx2id = {i:v for i,v in enumerate(matrix_train.columns)}

### NMF 분석

$X = ${User - Item 간의 value를 저장하는 matrix}

$X = R^{n_{user} \times n_{item}}$

In [113]:
X = matrix_train.values
display(pd.DataFrame(X, columns=matrix_train.columns).head())

assessmentItemID,A010001001,A010001002,A010001003,A010001004,A010001005,A010002001,A010002002,A010002003,A010002004,A010002005,...,A090073003,A090073004,A090073005,A090073006,A090074001,A090074002,A090074003,A090074004,A090074005,A090074006
0,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
1,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0
2,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
3,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
4,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5


위 matrix를 바탕으로 NMF 분석 수행

In [114]:
nmf = NMF(n_components=12)
nmf.fit(X)
Y = nmf.transform(X)

/opt/conda/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


추론을 위해 predict matrix 복원
 - 처음 pivot table의 값을 NMF로 구한 matrix를 통해 복원했을 때, 두 행렬 사이의 오차 (restore error) 는 0에 가까울수록 NMF가 올바르게 구해짐

In [115]:
X_pred = nmf.inverse_transform(Y)
restore_error = np.sum(np.square(X_pred - X)) /X_pred.size
print(f"Restore Error : {restore_error}")

Restore Error : 0.0076142635981724


## 🎯 예측 및 평가

### 학습 데이터 재현 평가

예측 함수 정의

In [116]:
def predict(userid, itemid):
    useridx = user_id2idx[userid]
    itemidx = item_id2idx[itemid]
    
    return X_pred[useridx, itemidx]

학습에 사용한 데이터를 얼마나 잘 예측하는지 평가

In [117]:
a_prob = [predict(u,i) for u,i in zip(train_data.userID, train_data.assessmentItemID)]
a_pred = [round(v) for v in a_prob] 
a_true = train_data.answerCode

print("Train data prediction")
print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")

Train data prediction
 - Accuracy = 76.15%
 - ROC-AUC  = 77.97%


In [118]:
print(len(a_prob))
a_prob[:5]

2220633


[0.722356553120757,
 0.7172303640519588,
 0.7022089416510257,
 0.7228440992214077,
 0.7093717094151408]

위 코드에서는 이미 학습된 사용자에 대해서만 추론값을 계산 가능하다.

- 테스트 데이터의 사용자는 학습 데이터셋에 존재하지 않는다.
- 따라서 해당 사용자의 값을 가져올 수 없기에 키 에러가 발생한다.

In [119]:
try:
    a_prob = [predict(u,i) for u,i in zip(test_data.userID, test_data.assessmentItemID)]
    a_pred = [round(v) for v in a_prob]
    a_true = test_data.answerCode

    print("Test data prediction")
    print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
    print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")
except:
    print("Error Occurs!!")

Error Occurs!!


### 테스트 데이터 재현 평가

학습되지 않은 사용자에 대해서도 문제를 푼 데이터가 존재할 경우 이를 바탕으로 추론 가능하다.

$B = $ {학습되지 않은 사용자에 대한 User - Item 간 value 행렬}

$ A = U \Sigma V$ 일때 factor matrx $A_{factor} = R^{n_{user} \times n_{factor}}$ 인 $A_{factor}$ 는
- $A_{factor} \Sigma V = A$
- $A_{factor} = A {(\Sigma V)}^+ = A V^T \Sigma^+  $ ($U, V$ 는 직교행렬)

$ B_{pred} \approx B_{factor} \Sigma V  = B V^T \Sigma^+ \Sigma V$

In [120]:
def predict(matrix, userid, itemid, user_id2idx, item_id2idx):
    X = matrix
    X_pred = nmf.inverse_transform(nmf.transform(X))

    ret = [X_pred[user_id2idx[u], item_id2idx[i]] for u,i in zip(userid, itemid)]
    return ret

학습 데이터 재현 성공률

In [121]:
a_prob = predict(matrix_train.values, train_data.userID, train_data.assessmentItemID, user_id2idx, item_id2idx)
a_true = train_data.answerCode
a_pred = [round(v) for v in a_prob]

print("Train data prediction")
print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")

Train data prediction
 - Accuracy = 76.15%
 - ROC-AUC  = 77.97%


In [122]:
print(len(a_prob))
a_prob[:5]

2220633


[0.722356553120757,
 0.7172303640519588,
 0.7022089416510257,
 0.7228440992214077,
 0.7093717094151408]

테스트 데이터 재현 성공률

In [123]:
# item_id2idx는 train에서 사용한 것을 다시 사용한다.
userid = sorted(list(set([u for u in test_data.userID])))
user_id2idx_test = {v:i for i,v in enumerate(userid)}

matrix_test = 0.5*np.ones((len(userid), len(item_id2idx)))
for user,item,a in zip(test_data.userID, test_data.assessmentItemID, test_data.answerCode):
    user,item = user_id2idx_test[user],item_id2idx[item]
    matrix_test[user,item] = a

# 성능 측정
a_prob = predict(matrix_test, test_data.userID, test_data.assessmentItemID, user_id2idx_test, item_id2idx)
a_true = test_data.answerCode
a_pred = [round(v) for v in a_prob] 

print("Test data prediction")
print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")

Test data prediction
 - Accuracy = 75.62%
 - ROC-AUC  = 77.29%


In [124]:
print(len(a_prob))
a_prob[:5]

254585


[0.6163606112193646,
 0.5326641227218643,
 0.5583177353868735,
 0.6361983806080543,
 0.5776611361699301]

### 테스트 평가 데이터 재현 평가

테스트 데이터 기반 선별된 평가항목 추론

In [125]:
a_prob = predict(matrix_test, eval_data.userID, eval_data.assessmentItemID, user_id2idx_test, item_id2idx)
a_true = eval_data.answerCode
a_pred = [round(v) for v in a_prob] 

print("Test data prediction")
print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")

Test data prediction
 - Accuracy = 71.64%
 - ROC-AUC  = 75.48%


In [126]:
print(len(a_prob))
a_prob[:5]

744


[0.5393966024487716,
 0.6371842657968889,
 0.44287857250366863,
 0.6672922671926579,
 0.558564435108218]

# Inference

In [127]:
def predict(matrix, userid, itemid, user_id2idx, item_id2idx):
    X = matrix
    X_pred = nmf.inverse_transform(nmf.transform(X))

    ret = [X_pred[user_id2idx[u], item_id2idx[i]] for u,i in zip(userid, itemid)]
    return ret

In [128]:
# item_id2idx는 train에서 사용한 것을 다시 사용한다.
userid = sorted(list(set([u for u in real_test_data.userID])))
user_id2idx_test = {v:i for i,v in enumerate(userid)}

matrix_test = 0.5*np.ones((len(userid), len(item_id2idx)))
for user,item,a in zip(real_test_data.userID, real_test_data.assessmentItemID, real_test_data.answerCode):
    user,item = user_id2idx_test[user],item_id2idx[item]
    matrix_test[user,item] = a

# 성능 측정
a_prob = predict(matrix_test, real_test_data.userID, real_test_data.assessmentItemID, user_id2idx_test, item_id2idx)
a_true = real_test_data.answerCode
a_pred = [round(v) for v in a_prob] 

print("Test data prediction")
print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")

Test data prediction
 - Accuracy = 75.61%
 - ROC-AUC  = 77.29%


In [129]:
print(len(a_prob))
a_prob[:5]

255329


[0.6164048536718825,
 0.5326233191683962,
 0.5583093755766838,
 0.6362453411900573,
 0.5776570647024208]

In [130]:
real_eval_data

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
1035,3,A050133008,A050000133,-1,2020-10-26 13:13:57,5289
1706,4,A070146008,A070000146,-1,2020-12-27 02:47:54,9080
3023,13,A070111008,A070000111,-1,2020-12-27 04:35:09,9660
4283,17,A090064006,A090000064,-1,2020-10-30 05:48:37,2611
4670,26,A060135007,A060000135,-1,2020-10-23 11:44:18,1422
...,...,...,...,...,...,...
260052,7395,A040122005,A040000122,-1,2020-09-08 02:05:20,10615
260067,7404,A030111005,A030000111,-1,2020-10-13 09:49:18,7636
260082,7416,A050193004,A050000193,-1,2020-10-04 02:44:41,10402
260097,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402


In [131]:
a_prob = predict(matrix_test, real_eval_data.userID, real_eval_data.assessmentItemID, user_id2idx_test, item_id2idx)
# a_true = eval_data.answerCode
a_pred = [round(v) for v in a_prob] 

print("Test data prediction")
# print(f" - Accuracy = {100*accuracy_score(a_true, a_pred):.2f}%")
# print(f" - ROC-AUC  = {100*roc_auc_score(a_true, a_prob):.2f}%")

Test data prediction


In [133]:
print(len(a_prob))
a_prob

744


[0.5704580396671269,
 0.649998864797505,
 0.4426949704319842,
 0.6440040530131592,
 0.4726912673480518,
 0.6081804724499869,
 0.5271250540972756,
 0.48213976461904245,
 0.4247165359699464,
 0.5506028113039302,
 0.5104467257220566,
 0.4845515088651715,
 0.6195170729959653,
 0.5022127162100354,
 0.6755839386957376,
 0.7381854691693692,
 0.48145104834205354,
 0.6228250284381223,
 0.6699852978678021,
 0.40396124325695126,
 0.7487728737486012,
 0.5154057555162987,
 0.4900021825447476,
 0.4758392549186678,
 0.48681692014036665,
 0.5467982614996896,
 0.5438521433031118,
 0.7306030188259444,
 0.508350747668556,
 0.6832894009550682,
 0.6159759314738802,
 0.5484663047537789,
 0.5290298737125636,
 0.499381227284189,
 0.6949670308387393,
 0.7328360494895836,
 0.4503092374109669,
 0.4982229866077885,
 0.5648344378298843,
 0.5025490164043676,
 0.5115644191792764,
 0.35966842111849084,
 0.3738232866978977,
 0.4388942293244055,
 0.5500344330056585,
 0.517757010979133,
 0.6313163914835414,
 0.408794445

In [137]:
data=list()
for id in range(len(a_prob)):
    data.append([id,a_prob[id]])

submit=pd.DataFrame(columns=['id','prediction'], data=data)
submit.head()

,id,prediction
0,0,0.570458
1,1,0.649999
2,2,0.442695
3,3,0.644004
4,4,0.472691


In [138]:
# SAVE OUTPUT
output_dir = "output/"
write_path = os.path.join(output_dir, "submission.csv")
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
submit.to_csv(write_path)